# `ConditionalDimension` tutorial

This tutorial explains how to create equations that only get executed under given conditions.

As in all our tutorials, we begin by specifying some parameters.
For our examples we will use `Function`s defined on 2D grids and their data values are going to be updated according to prescribed conditions.

In [1]:
#NBVAL_IGNORE_OUTPUT

from devito import Dimension, Function, Grid
import numpy as np

# We define a 10x10 grid, dimensions are x, y
shape = (10, 10)
grid = Grid(shape = shape)
x, y = grid.dimensions

# Define function 𝑓. We will populate f's data with ones on its diagonal.

f = Function(name='f', grid=grid)
f.data[:, :] = np.eye(10)
f.data

Data([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

Now lets construct a simple `Operator` to increase the values of $ f $ by 1.

In [2]:
#NBVAL_IGNORE_OUTPUT

from devito import Eq, Operator
op = Operator(Eq(f, f + 1))
op.apply()
f.data

Operator `Kernel` run in 0.01 s


Data([[2., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      [1., 2., 1., 1., 1., 1., 1., 1., 1., 1.],
      [1., 1., 2., 1., 1., 1., 1., 1., 1., 1.],
      [1., 1., 1., 2., 1., 1., 1., 1., 1., 1.],
      [1., 1., 1., 1., 2., 1., 1., 1., 1., 1.],
      [1., 1., 1., 1., 1., 2., 1., 1., 1., 1.],
      [1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
      [1., 1., 1., 1., 1., 1., 1., 2., 1., 1.],
      [1., 1., 1., 1., 1., 1., 1., 1., 2., 1.],
      [1., 1., 1., 1., 1., 1., 1., 1., 1., 2.]], dtype=float32)

Every value has been updated by one. You should be able to see twos on the diagonal and ones everywhere else.

In [3]:
#NBVAL_IGNORE_OUTPUT
print(op.ccode) # Print the generated code

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict f_vec, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]]) f_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(1) schedule(static,1)
    for (int x = x_m; x <= x_M; x += 1)
    {

# Devito API for relationals

In order to construct a `ConditionalDimension`, one needs to get familiar with relationals. The Devito API for relationals currently supports the following classes of relation, which inherit from their [SymPy counterparts](https://docs.sympy.org/latest/modules/core.html#module-sympy.core.relational):

- Le (less-than or equal)
- Lt (less-than)
- Ge (greater-than or equal)
- Gt (greater-than)
- Ne (not equal)

Relationals are used to define a condition and are passed as an argument to `ConditionalDimension` at construction time.


# Devito API for ConditionalDimension

A `ConditionalDimension` defines a non-convex iteration sub-space derived from a ``parent`` Dimension, implemented by the compiler generating conditional "if-then" code within the parent Dimension's iteration space.

A `ConditionalDimension` is used in two typical cases: 

- Use case A:
To constrain the execution of loop iterations to make sure certain conditions are honoured

- Use case B:
To subsample a `Function`

In [4]:
from devito import ConditionalDimension
print(ConditionalDimension.__doc__)


    Symbol defining a non-convex iteration sub-space derived from a ``parent``
    Dimension, implemented by the compiler generating conditional "if-then" code
    within the parent Dimension's iteration space.

    Parameters
    ----------
    name : str
        Name of the dimension.
    parent : Dimension
        The parent Dimension.
    factor : int, optional
        The number of iterations between two executions of the if-branch. If None
        (default), ``condition`` must be provided.
    condition : expr-like, optional
        An arbitrary SymPy expression, typically involving the ``parent``
        Dimension. When it evaluates to True, the if-branch is executed. If None
        (default), ``factor`` must be provided.
    indirect : bool, optional
        If True, use ``condition``, rather than the parent Dimension, to
        index into arrays. A typical use case is when arrays are accessed
        indirectly via the ``condition`` expression. Defaults to False.

    Examp

# Use case A (honour a certain condition)


## Defining a condition
Now it's time to show a more descriptive example. Let's define a conditional that updates the values of the `Function` that are larger than zero, f(x,y) > 0. We name our `ConditionalDimension` `ci`.
In this example we want to update again by one all the elements in `f` that are larger than zero, $f(x,y) > 0$
Before updating the elements we initialize our data again to the eye function.

In [5]:
#NBVAL_IGNORE_OUTPUT

from devito.types import Gt

f.data[:, :] = np.eye(10)

# Define the condition f(x, y) > 0
condition = Gt(f, 0)

# Construct a ConditionalDimension
ci = ConditionalDimension(name='ci', parent=y, condition=condition)

op = Operator(Eq(f, f + 1))
op.apply()
f.data
# print(op.ccode) # Uncomment to view code


Operator `Kernel` run in 0.01 s


Data([[2., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      [1., 2., 1., 1., 1., 1., 1., 1., 1., 1.],
      [1., 1., 2., 1., 1., 1., 1., 1., 1., 1.],
      [1., 1., 1., 2., 1., 1., 1., 1., 1., 1.],
      [1., 1., 1., 1., 2., 1., 1., 1., 1., 1.],
      [1., 1., 1., 1., 1., 2., 1., 1., 1., 1.],
      [1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
      [1., 1., 1., 1., 1., 1., 1., 2., 1., 1.],
      [1., 1., 1., 1., 1., 1., 1., 1., 2., 1.],
      [1., 1., 1., 1., 1., 1., 1., 1., 1., 2.]], dtype=float32)

If we look at the generated code, we will see that it is identical to the one from cell 3 -- no conditionals are generated. Why? We can notice that there are no conditionals in the generated code and as expected the function values are not updated. But why? We notice that `ci` is not used anywhere after construction. Neither `f` nor `op` know about `ci` yet. We then need to pass `ci` explicitly as an "implicit `Dimension`" to the equation that we want to be conditionally executed as shown in the next cell.

In [6]:
#NBVAL_IGNORE_OUTPUT

f.data[:, :] = np.eye(10)

op = Operator(Eq(f, f + 1, implicit_dims=ci))
print(op.ccode)
op.apply()

assert (np.count_nonzero(f.data - np.diag(np.diagonal(f.data)))==0)
assert (np.count_nonzero(f.data) == 10)

f.data

Operator `Kernel` run in 0.01 s


#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict f_vec, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]]) f_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(1) schedule(static,1)
    for (int x = x_m; x <= x_M; x += 1)
    {

Data([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 2., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]], dtype=float32)

Now the generated code is as expected and only the elements that were greater than zero were incremented by one.


Let's now create a new `Function` `g`, initialized with 1's along its diagonal.
Let's try to add `f` and `g` only if the elements in `g` are not equal to zero and only for the first five rows. Now, we need to satisfy two conditions so we will additionally use SymPy's `And`.

In [7]:
#NBVAL_IGNORE_OUTPUT

from sympy import And
from devito.types import Ne, Lt

f.data[:,:] = np.eye(10)


g = Function(name='g', grid=grid)
g.data[:,:] = np.eye(10)

ci = ConditionalDimension(name='ci', parent=y, condition=And(Ne(g, 0), Lt(y, 5)))

op = Operator(Eq(f, f + g, implicit_dims=ci))
op.apply()

print(op.ccode)

assert (np.count_nonzero(f.data - np.diag(np.diagonal(f.data)))==0)
assert (np.count_nonzero(f.data) == 10)
assert np.all(f.data[np.nonzero(f.data[:5,:5])] == 2)
assert np.all(f.data[5:,5:] == np.eye(5))

f.data

Operator `Kernel` run in 0.01 s


#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict f_vec, struct dataobj *restrict g_vec, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]]) f_vec->data;
  float (*restrict g)[g_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[g_vec->size[1]]) g_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
 

Data([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

You can see that now `f` has been updated only for the first five rows with the `f+g` expression.

A `ConditionalDimension` can be also used at the construction time of a `Function`. Let's use `ci` from the previous cell to explicitly construct function `h`. You will notice that now there is no need to pass argument `implicit_dims` as function's `h`s dimensions include the `ConditionalDimension` `ci`. `h` still has the size of the full domain, not the size of the domain that satisfies the condition.

In [8]:
#NBVAL_IGNORE_OUTPUT

h = Function(name='h', shape=grid.shape, dimensions=(x, ci))

op = Operator(Eq(h, h + g))
op.apply()

print(op.ccode)

assert (np.count_nonzero(h.data) == 5)

h.data

Operator `Kernel` run in 0.01 s


#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict g_vec, struct dataobj *restrict h_vec, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m)
{
  float (*restrict g)[g_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[g_vec->size[1]]) g_vec->data;
  float (*restrict h)[h_vec->size[1]] __attribute__ ((aligned (64))) = (float (*)[h_vec->size[1]]) h_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  for (int x = x_m; x

Data([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

# Use case B (Function subsampling)

`ConditionalDimension`s are additionally indicated to implement `Function` subsampling. In the following example, an `Operator` subsamples `Function` `g` and saves its content into `f` every `factor=4` iterations.

In [9]:
#NBVAL_IGNORE_OUTPUT
size, factor = 16, 4
i = Dimension(name='i')
ci = ConditionalDimension(name='ci', parent=i, factor=factor)

g = Function(name='g', shape=(size,), dimensions=(i,))
# Intialize g 
g.data[:,]= list(range(size))
f = Function(name='f', shape=(int(size/factor),), dimensions=(ci,))

op = Operator([Eq(f, g)])
print(op.ccode)

op.apply()

assert np.all(f.data[:] == g.data[0:-1:4])

print("\n Data in g \n", g.data)
print("\n Subsampled data in f \n", f.data)

Operator `Kernel` run in 0.01 s


#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict f_vec, struct dataobj *restrict g_vec, const int i_M, const int i_m, struct profiler * timers, const int nthreads)
{
  float (*restrict f) __attribute__ ((aligned (64))) = (float (*)) f_vec->data;
  float (*restrict g) __attribute__ ((aligned (64))) = (float (*)) g_vec->data;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(1) schedule(stati

Finally, we can notice that `f` holds as expected the subsampled values of `g`.

# References

- [ConditionalDimension](https://github.com/devitocodes/devito/blob/e918e26757a4f0ea3c3c416b6e0f48f4e5023c37/devito/types/dimension.py#L635 "class ConditionalDimension(DerivedDimension):") documentation.